In [24]:
using BSON: @load
using Flux
using Flux: chunk
using Flux.Data: DataLoader
using ImageFiltering
using Images
using ImageIO
using MLDatasets: FashionMNIST
using LinearAlgebra
using MLDatasets
using Plots
using Zygote
using FFTW
using Distributions
using SparseArrays

In [4]:
function PLUGIn_CS(G, y, A, max_iter, stepsize, tolerance, out_toggle)
    
    (_, z_dim) = size(Flux.params(G[1])[1]);
    W = I(z_dim)
  
    #normalize the weights of the network
    for i in 1:length(G)
        _, s, _ = svd(Flux.params(G[i])[1])
        W = Flux.params(G[i])[1] * W /s[1]
    end
  
    z = randn(z_dim)
    iter = 1
    succ_error = 1
  
    while iter <= max_iter && succ_error > tolerance
      
      # d gives the PLUGIn direction
      d = W'*A'*(A * G(z) - y)
      z -= stepsize * d
      succ_error = norm(stepsize * d)
      if iter % out_toggle == 0  
          println("====> In quasi-gradient: Iteration: $iter Successive error: $succ_error")
      end
      iter += 1
    end
    println("====> In quasi-gradient: Iteration: $iter Successive error: $succ_error")
  
    return z
  end

PLUGIn_CS (generic function with 1 method)

In [5]:
function GD_CS(G, y, A, max_iter, stepsize, tolerance, out_toggle)

    z = randn(20)
    iter = 1
    succ_error = 1
    while iter <= max_iter && succ_error > tolerance
        # d gives the PLUGIn direction
        d = gradient(z -> norm(y - A*G(z)), z)[1]
        z -= stepsize * d
        succ_error = norm(stepsize * d)
        if iter % out_toggle == 0  
            println("====> In quasi-gradient: Iteration: $iter Successive error: $succ_error")
        end
        iter += 1
    end
    println("====> In quasi-gradient: Iteration: $iter Successive error: $succ_error")

    return z
end

GD_CS (generic function with 1 method)

In [6]:
#setup a synthetic problem
G = Chain(
    Dense(20, 500, relu, bias = false; initW =(out,in) ->  randn(500, 20)/sqrt(500)),
    Dense(500, 500, relu, bias = false; initW =(out,in) -> randn(500, 500)/sqrt(500)),
    Dense(500, 784, relu, bias = false; initW =(out,in) -> randn(784, 500)/sqrt(784))
)


z = randn(20)
m = 300; A = randn(m, 784)/sqrt(m)
y = A*G(z) + 1e-14 * randn(m)

stepsize = 2
tolerance = 1e-14
max_iter = 10000
out_toggle = 1000
z_rec = PLUGIn_CS(G,y,A, max_iter, stepsize, tolerance, out_toggle)
recov_error = norm(z - z_rec)
recon_error = norm(G(z) - G(z_rec))
println("recovery error: $recov_error, reconstruction error: $recon_error")

====> In quasi-gradient: Iteration: 1000 Successive error: 1.9656888979299607e-5
====> In quasi-gradient: Iteration: 2000 Successive error: 5.5364298995732266e-9
====> In quasi-gradient: Iteration: 3000 Successive error: 1.3135294395789192e-12
====> In quasi-gradient: Iteration: 3629 Successive error: 9.937721023672761e-15
recovery error: 5.613198580172336e-13, reconstruction error: 1.612694150932187e-13


Compresed sensing with Fourier measurements

In [25]:
x_dim = 100
sampling = rand(Bernoulli(.5), x_dim)
sampling[1] = 1
m = sum(sampling)
F = fft(Diagonal(sampling), 2)


# x = zeros(x_dim); x[1] = 1
# fft(x)

findnz(sampling)



LoadError: MethodError: no method matching findnz(::Vector{Bool})
[0mClosest candidates are:
[0m  findnz([91m::SparseArrays.AbstractSparseMatrixCSC{Tv, Ti}[39m) where {Tv, Ti} at /cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/julia/1.7.0/share/julia/stdlib/v1.7/SparseArrays/src/sparsematrix.jl:1538
[0m  findnz([91m::SparseVector{Tv, Ti}[39m) where {Tv, Ti} at /cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/julia/1.7.0/share/julia/stdlib/v1.7/SparseArrays/src/sparsevector.jl:783
[0m  findnz([91m::AbstractSparseMatrix[39m) at /project/def-mpf/bjoshi/.julia/packages/Plots/hxZ0l/src/recipes.jl:1610
[0m  ...

In [26]:
?findnz

search: findnz findnext findmin! findmin findlocalminima



```
findnz(A::SparseMatrixCSC)
```

Return a tuple `(I, J, V)` where `I` and `J` are the row and column indices of the stored ("structurally non-zero") values in sparse matrix `A`, and `V` is a vector of the values.

# Examples

```jldoctest
julia> A = sparse([1 2 0; 0 0 3; 0 4 0])
3×3 SparseMatrixCSC{Int64, Int64} with 4 stored entries:
 1  2  ⋅
 ⋅  ⋅  3
 ⋅  4  ⋅

julia> findnz(A)
([1, 1, 3, 2], [1, 2, 2, 3], [1, 2, 4, 3])
```

---

```
findnz(x::SparseVector)
```

Return a tuple `(I, V)`  where `I` is the indices of the stored ("structurally non-zero") values in sparse vector `x` and `V` is a vector of the values.

# Examples

```jldoctest
julia> x = sparsevec([1 2 0; 0 0 3; 0 4 0])
9-element SparseVector{Int64, Int64} with 4 stored entries:
  [1]  =  1
  [4]  =  2
  [6]  =  4
  [8]  =  3

julia> findnz(x)
([1, 4, 6, 8], [1, 2, 4, 3])
```


In [27]:
# experiments with MNIST dataset
function load_model(load_dir::String, epoch::Int)
    print("Loading model...")
    @load joinpath(load_dir, "model-$epoch.bson") encoder_μ encoder_logvar decoder
    println("Done")
    return encoder_μ, encoder_logvar, decoder
end

function get_train_loader(batch_size, shuffle::Bool)
    # The MNIST training set is made up of 60k 28 by 28 greyscale images
    train_x, train_y = MNIST.traindata(Float32)
    train_x = 1 .- reshape(train_x, (784, :))
    return DataLoader((train_x, train_y), batchsize=batch_size, shuffle=shuffle, partial=false)
end

epoch_to_load = 20
# Load the model and test set loader
encoder_mu, encoder_logvar, decoder = load_model("result", epoch_to_load)

# pick a image in MNIST to denoise
num = 8
batch_size = 64
shuffle_data = true
dataloader = get_train_loader(batch_size, shuffle_data)

(x_batch, y_batch) = first(dataloader)
i = 1
while y_batch[i] != num
    i += 1
end

x = x_batch[:,i]
noise_level = .1

m = 300; A = randn(m, 784)/sqrt(m)

y = A*x + noise_level * randn(m)

stepsize = 1
tolerance = 1e-7
max_iter = 10000
out_toggle = 1000
z_rec_PLUGIn = PLUGIn_CS(decoder, y, A, max_iter, stepsize, tolerance, out_toggle)
error = norm(x - decoder(z_rec_PLUGIn))
println("reconstruction error: $error")

z_rec_GD = GD_CS(decoder, y, A, max_iter, stepsize, tolerance, out_toggle)

recovered_image_PLUGIn = colorview(Gray, reshape(decoder(z_rec_PLUGIn), 28,28)' )
recovered_image_GD = colorview(Gray, reshape(decoder(z_rec_GD), 28,28)' )

true_image = colorview(Gray, reshape(x, 28,28)' );
p1 = plot(true_image, framestyle = :none, bg =:black, title = "original image")
p2 = plot(recovered_image_PLUGIn, framestyle = :none, bg =:black, title = "recovered image PLUGIn,\n m = $m")
p3 = plot(recovered_image_GD, framestyle = :none, bg =:black, title = "recovered image GD,\n m = $m")
plot(p1, p2, p3, layout = 3)


Loading model...

LoadError: SystemError: opening file "result/model-20.bson": No such file or directory